In [9]:
import time
from datetime import date
import hmac
from requests import Request, Session
import pandas as pd
import numpy as np

#API
def api(prepared):
    ts = int(time.time() * 1000)
    signature_payload = f'{ts}{prepared.method}{prepared.path_url}'.encode()
    signature = hmac.new('SECRET_KEY'.encode(), signature_payload, 'sha256').hexdigest()#シークレットキー
    prepared.headers['FTX-KEY'] = 'API_KEY'#APIキー
    prepared.headers['FTX-SIGN'] = signature
    prepared.headers['FTX-TS'] = str(ts)
    return prepared

#全銘柄
def crypts():
    prepared = Request('GET', 'https://ftx.com/api/markets').prepare()
    Api = api(prepared)
    json = Session().send(Api).json()['result']
    
    df = pd.DataFrame(json)
    df = df.loc[:, ['name']]#826
    crs = df.query('name.str.endswith("/USD") | name.str.endswith("/JPY")', engine='python')#427 
    crs = crs.query('(~name.str.contains("BEAR") & ~name.str.contains("BULL") & ~name.str.contains("HALF") & ~name.str.contains("HEDGE"))', engine='python')#260
    return crs['name']

#データ(close, volume)
def cr_df(crypt):
    prepared = Request('GET', 'https://ftx.com/api/markets/'+crypt+'/candles?resolution=3600').prepare()
    Api = api(prepared)
    json = Session().send(Api).json()['result']
    
    df = pd.DataFrame(json)
    df = df[['startTime', 'close', 'volume']]
    df['startTime'] = pd.to_datetime(df['startTime'])
    df = df.rename(columns = {'startTime' : 'date'})
    df = df.set_index('date')
    return df

#取引量上位銘柄データ（数=cr_num）
def df(crypts, cr_num):
    li_vol = []
    li_price = []
    for crypt in crypts:
        try:
            Cr_df = cr_df(crypt)
            ave = np.average(Cr_df['volume'])
            vol = pd.DataFrame({'name' : [crypt], 'vol_ave' : [ave]})
            li_vol.append(vol)
        
            Cr_df = Cr_df.drop(['volume'], axis=1)
            price = Cr_df.rename(columns = {'close' : crypt})
            li_price.append(price)
        except(KeyError):
            continue
        
    df_vol = pd.concat(li_vol)
    Lcr = df_vol.nlargest(cr_num, "vol_ave", keep="all")['name']
    
    df_price = pd.concat(li_price, axis=1)
    df = df_price[Lcr]
    return df

#上位10銘柄データ
class Main_FTX:
    
    def __init__(self, crypts):
        self.Crypts = crypts
        
    def result():
        Crypts = crypts()
        Df = df(Crypts, 10)
    

Main_FTX()

SyntaxError: invalid syntax (2983088589.py, line 70)

In [32]:
import time
from datetime import date
import hmac
from requests import Request, Session
import pandas as pd
import numpy as np

class Main_FTX:
    def __init__(self, num):
        self.num = num

#API
    def api(prepared):
        ts = int(time.time() * 1000)
        signature_payload = f'{ts}{prepared.method}{prepared.path_url}'.encode()
        signature = hmac.new('SECRET_KEY'.encode(), signature_payload, 'sha256').hexdigest()#シークレットキー
        prepared.headers['FTX-KEY'] = 'API_KEY'#APIキー
        prepared.headers['FTX-SIGN'] = signature
        prepared.headers['FTX-TS'] = str(ts)
        return prepared
    
    #全銘柄
    def crypts():
        prepared = Request('GET', 'https://ftx.com/api/markets').prepare()
        Api = api(prepared)
        json = Session().send(Api).json()['result']
    
        df = pd.DataFrame(json)
        df = df.loc[:, ['name']]#826
        crs = df.query('name.str.endswith("/USD") | name.str.endswith("/JPY")', engine='python')#427 
        crs = crs.query('(~name.str.contains("BEAR") & ~name.str.contains("BULL") & ~name.str.contains("HALF") & ~name.str.contains("HEDGE"))', engine='python')#260
        return crs['name']
    
    #データ(close, volume)
    def cr_df(crypt):
        prepared = Request('GET', 'https://ftx.com/api/markets/'+crypt+'/candles?resolution=3600').prepare()
        Api = api(prepared)
        json = Session().send(Api).json()['result']

        df = pd.DataFrame(json)
        df = df[['startTime', 'close', 'volume']]
        df['startTime'] = pd.to_datetime(df['startTime'])
        df = df.rename(columns = {'startTime' : 'date'})
        df = df.set_index('date')
        return df

    #取引量上位銘柄データ（数=cr_num）
    def df(crypts, self):
        li_vol = []
        li_price = []
        for crypt in crypts:
            try:
                Cr_df = cr_df(crypt)
                ave = np.average(Cr_df['volume'])
                vol = pd.DataFrame({'name' : [crypt], 'vol_ave' : [ave]})
                li_vol.append(vol)

                Cr_df = Cr_df.drop(['volume'], axis=1)
                price = Cr_df.rename(columns = {'close' : crypt})
                li_price.append(price)
            except(KeyError):
                continue

        df_vol = pd.concat(li_vol)
        Lcr = df_vol.nlargest(self.num, "vol_ave", keep="all")['name']
    
        df_price = pd.concat(li_price, axis=1)
        df = df_price[Lcr]
        return df
    
    def result():
        Crypts = crypts()
        Df = df(Crypts, 10)
        print(Df)
    


type(Main_FTX(10))

__main__.Main_FTX